<a href="https://colab.research.google.com/github/jvishnuvardhan/Stackoverflow_Questions/blob/master/Custom_Layer_with_Functional_API_saving_and_loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==2.2.0rc4

In [0]:
    def get_config(self):
      config = super(CustomLayer, self).get_config()
      config.update({'add_loss': self.add_loss})
      return config

In [0]:
import numpy as np
import tensorflow as tf


class CustomLayer(tf.keras.layers.Layer):
    """Imaginary Layer that adds a custom loss in the call"""

    def __init__(self, a):
        super().__init__()
        self.var = tf.Variable(a, name='var_a')

    def call(self, inputs, training=False):
      output = tf.reduce_sum(inputs * self.var, axis=-1)
      self.add_loss(tf.reduce_mean(output))
      return output
    def get_config(self):
      return {'a': self.var.numpy()}

def get_model(input_dim: int) -> tf.keras.Model:
    layer = CustomLayer(0.1)
    inputs = tf.keras.Input((input_dim,), name="inputs")
    outputs = layer(inputs)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam')
    return model


if __name__ == "__main__":
    num_data = 100
    X, Y = np.random.randn(num_data, 1), np.random.randn(num_data, 1)

    model = get_model(input_dim=X.shape[-1])
    
    model.summary()
    print("model.losses", model.losses)
    model.fit(X,Y, epochs=2)
    model.save('my_model',save_format='tf')
    print('-'*65)
    print('\n-------------- After loading the model back -----------------\n')
    print('-'*65)
    reconstructed_model = tf.keras.models.load_model('my_model',custom_objects={'CustomLayer':CustomLayer})  # ~~ breaks 
    reconstructed_model.summary()

    # Let's check:
    np.testing.assert_allclose(
        model.predict(X),
        reconstructed_model.predict(X)
    )

    print("model.losses:", model.losses)
    print("reconstructed_model.losses:", reconstructed_model.losses)

    reconstructed_model.fit(X,Y, epochs=2)

Model: "model_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 1)]               0         
_________________________________________________________________
custom_layer_43 (CustomLayer (None,)                   1         
Total params: 1
Trainable params: 1
Non-trainable params: 0
_________________________________________________________________
model.losses [<tf.Tensor 'custom_layer_43/Mean:0' shape=() dtype=float32>]
Epoch 1/2
4/4 [==============================] - 0s 2ms/step - loss: 0.0055
Epoch 2/2
4/4 [==============================] - 0s 2ms/step - loss: 0.0038
INFO:tensorflow:Assets written to: my_model/assets
-----------------------------------------------------------------

-------------- After loading the model back -----------------

-----------------------------------------------------------------
Model: "model_29"
__________________________________